<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/master/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Enunciado](https://www.u-cursos.cl/ingenieria/2020/1/CC3001/1/tareas/r/1_CC3001-Tarea2-Diagrama_de_estados.pdf)

In [18]:
dic = {}

def leerlinea(str):
  linea = ''
  for char in str:
    linea += char
  return linea

def asignar(linea):
  nombrevar = ''
  valor = ''
  i = 0
  while i < len(linea):
    if linea[i] == '=':
      valor = '' 
      i += 1
    elif linea[i] == '_' or esInt(linea[i]) or esAlfa(linea[i]):
      nombrevar += linea[i]
      i += 1
    else:
      valor += linea[i]
      i += 1
  valor = int(valor)
  dic[nombrevar] = valor
  return None

def procesar(linea):
  if '=' in linea:
    if '+' in linea: pass
    else: asignar(linea)
  else:
    printear(linea)

def esInt(char):
  Enteros = ['0', '1', '2', '3', '4', '5', '6', '7', '8' ,'9']
  b = char in Enteros
  return b

def esAlfa(char):
  Letras = ['a','b','c','d','e','f','g',\
            'h','i','j','k','l','m','n',\
            'o','p','q','r','s','t','u',\
            'v','w','x','y','z',\
            'A', 'B', 'C', 'D', 'E', 'F',\
            'G', 'H', 'I', 'J', 'K', 'L', \
            'M', 'N', 'O', 'P', 'Q', 'R', \
            'S', 'T', 'U', 'V', 'W', 'X', \
            'Y', 'Z']
  b = char in Letras
  return b

def esOp(char):
  Operaciones = ['+','-','*','/','^']

def printear(str):
  print(dic[str])

while True:
  a = input()
  if a == 'salir': break
  a = leerlinea(a)
  procesar(a)

dfhsagjhkjdlsa___872193hedohe983nd__7yeh739 = 637281687


ValueError: ignored